# Good practices in Julia
by Ludmilla Figueiredo & Victor Boussange

Here, we present a couples of tools to implement good programming and research practices without increasing your workflow too much.


## Setting up

Having a basic, preferrably automated, set up from where to start your work can be very helpful saving time because:
- you can start your project right away
- you have a consistent (familiar) structure where you locate yourself faster, both mentally and "programmatically"

With that in mind, I developed this [starter-kit to use so called "computational notebooks" to structure your work](https://github.com/FellowsFreiesWissen/computational_notebooks/tree/file_struct2)

Here, we will work with the following file structure:

<img src="file_structure_second_branch.png" alt="File structure created by `set_kit.jl`" width="400" height="auto">

Extra files not shown above, but typical for Julia projects:
- [`Project.toml`*](https://github.com/vboussange/iDiv-Julia-Workshop-2024/blob/main/Day1/22_pkg/Pkg.ipynb)
- [`Manifest.toml`*](https://github.com/vboussange/iDiv-Julia-Workshop-2024/blob/main/Day1/22_pkg/Pkg.ipynb)
- `run_code.sh`: shell script you can use to run your julia scripts (at the end of this page).
- `.gitignore`: specifies which files and directories should be ignored by version control.

`Project.toml` and `Manifest.toml` are created when you have dependencies in your project, i.e. when you use packages; you don't have to necessarily create them. It is, however, possible to start the same environment of another project from its `Project.toml` and `Manifest.toml` files, with `Pkg.instante`. Therefore, tracking your `Project.toml` and `Manifest.toml` is essential for the reproducibility of your work.


To set this up, **run the following from the Julia REPL**:

```julia
include("your_path_to/set_kit.jl")
set_kit("where_your_project_should_be", "name_of_project")
```

# Testing your code


Use the [Test-Driven Development (TDD) principles](https://en.wikipedia.org/wiki/Test-driven_development) to guide your development workflow and your life will be easier. Not only because your code will be more robust, but because adding tests after you have your finished product is not fun (Trust me, ~~I have had to add ad hoc tests and it was hell~~ I'm a scientist).


## Unit testing


- Test for correctness with typical inputs.
- Test edge cases.
- Test for errors with bad inputs.

A good idea is to write an additional test **whenever you find a bug in your code**. You kill two birds with one stone: solve your bug and increase the code coverage of your tests (which is a valuable feature of your code).


### Lightweight formal tests with `assert`

The simplest form of unit testing involves some sort of `assert` statement.


In [1]:
# import Pkg; Pkg.add("Test"); Pkg.add("Distributions")
using Test
using Random, Distributions

In [29]:
@assert 1 == 0 "The numbers are not the same"

AssertionError: AssertionError: The numbers are the same

In [30]:
@assert 1 == 1 "The numbers are not the same"


In practice, you would directly place the `assert` statement after your functions (or any bit of critical computing). This way, basic tests are run each time you execute the script. 


In [14]:
function create_varname(str)
    join(split(lowercase(str), " "), "_")
end

@assert create_varname("var Name") == "var_name" "Couldn't convert your variable's name"

Beware that the assertion error message will be limited in its ability to inform of why the test failed (more about it in the next section). Below, two tests failed, but `@assert` would not be able to distinguish them.

In [31]:
@assert create_varname("VarName") == "var_name" "Couldn't convert your variable's name"

AssertionError: AssertionError: Your variable is one-word

In [34]:
@assert create_varname("Variable_2") == "var_name" "Couldn't convert your variable's name"

AssertionError: AssertionError: Couldn't convert your variable's name

Use `isapprox` for floating point comparisons. (Beware that `rtol = (a-b)/max(abs(a), abs(b))`.)

In [2]:
function calculate_pi(diam, radius)
    @assert isapprox(2*3.14*radius, diam, rtol=0.2)  "Diameter value is 20% too big or too small" 
    pi = diam/(2*radius)
    return pi
end

calculate_pi (generic function with 1 method)

In [3]:
calculate_pi(10,5)

AssertionError: AssertionError: Diameter value is 20% too big or too small

In [5]:
calculate_pi(31.4, 5)

3.1399999999999997

In [6]:
calculate_pi(31.41592,5)

3.141592

In [10]:
@assert isapprox(calculate_pi(2*3.1*5, 5), π, atol=0.01)  "pi value used is 0.01 smaller/bigger than π" 

AssertionError: AssertionError: pi value used is 0.01 smaller/bigger than π


### Testing with Test.jl

For more complex functions and testing, you can use the `Test` module, relying on the `@test` macro. For cleaner code, it is recommended that you place your tests in `test/runtests.jl`.

Use `@testset` to group related tests together and a informative output:

In [13]:
@testset "Valid input" begin
    @test isapprox(calculate_pi(2*3.141*5, 5), π, atol=0.01)
    @test isapprox(calculate_pi(2*3.1415*5, 5), π, atol=0.01)
    @test isapprox(calculate_pi(2*3.14159*5, 5), π, atol=0.01)
end;

Test Summary: | Pass  Total  Time
Valid input   |    3      3  0.0s


In [15]:
@testset "Valid input" begin
    @test isapprox(calculate_pi(2*3.141*5, 5), π, atol=0.01)
    @test isapprox(calculate_pi(2*3.1415*5, 5), π, atol=0.01)
    @test isapprox(calculate_pi(2*3.14159*5, 5), π, atol=0.01)
    @test isapprox(calculate_pi(2*3.1*5, 5), π, atol=0.01)
end;

Valid input: Test Failed at c:\Users\fx97izoz\Documents\iDiv\julia_course\iDiv-Julia-Workshop-2024\Day1\23_project_management\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X52sZmlsZQ==.jl:5
  Expression: isapprox(calculate_pi(2 * 3.1 * 5, 5), π, atol = 0.01)
   Evaluated: isapprox(3.1, π; atol = 0.01)

Stacktrace:
 [1] macro expansion
   @ C:\Users\fx97izoz\AppData\Local\Programs\Julia-1.9.0-rc2\share\julia\stdlib\v1.9\Test\src\Test.jl:478 [inlined]
 [2] macro expansion
   @ c:\Users\fx97izoz\Documents\iDiv\julia_course\iDiv-Julia-Workshop-2024\Day1\23_project_management\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X52sZmlsZQ==.jl:5 [inlined]
 [3] macro expansion
   @ C:\Users\fx97izoz\AppData\Local\Programs\Julia-1.9.0-rc2\share\julia\stdlib\v1.9\Test\src\Test.jl:1498 [inlined]
 [4] top-level scope
   @ c:\Users\fx97izoz\Documents\iDiv\julia_course\iDiv-Julia-Workshop-2024\Day1\23_project_management\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X52sZmlsZQ==.jl:2
Test Summ

TestSetException: Some tests did not pass: 3 passed, 1 failed, 0 errored, 0 broken.

#### Testing your package/project

If you organize your work as a package, you can test it after activating your environment:

```jl
import Test, Pkg
Pkg.activate .
Test.test MyProject
```

### Continuous integration (CI)

 
- Automatically run tests on each commit/push. 

- [GitHub Actions](https://docs.github.com/en/actions/about-github-actions/about-continuous-integration-with-github-actions) is free and available within GitHub. It is rather straight forward to use. Other tools are Travis and AppVeyor (not fully working for Julia yet).

- GitHub Actions can also build documentation, check for code coverage, and [more](https://github.com/r-lib/actions/tree/v2/examples).

- CI is based on `.yaml` files, which specify the environment to run the testing script. GitHub Actions has [examples for many languages](https://docs.github.com/en/actions/use-cases-and-examples). Unfortunately not yet for Julia, but the [community is working on it](https://github.com/julia-actions).

Here is an example of a `.yaml` that will test your Julia package on different operating systems (linus, mac and Windows):

```yaml
name: Run tests

on:
  push:
    branches:
      - master
      - main
  pull_request:

permissions:
  actions: write
  contents: read

jobs:
  test:
    runs-on: ${{ matrix.os }}
    strategy:
      matrix:
        julia-version: ['1.6', '1', 'nightly']
        julia-arch: [x64, x86]
        os: [ubuntu-latest, windows-latest, macOS-latest]
        exclude:
          - os: macOS-latest
            julia-arch: x86

    steps:
      - uses: actions/checkout@v4
      - uses: julia-actions/setup-julia@v1
        with:
          version: ${{ matrix.julia-version }}
          arch: ${{ matrix.julia-arch }}
      - uses: julia-actions/cache@v1
      - uses: julia-actions/julia-buildpkg@v1
      - uses: julia-actions/julia-runtest@v1
```



### Other types of tests

- [**Docstring tests**](https://documenter.juliadocs.org/stable/man/doctests/): Unit tests embedded in docstrings.
- **Integration tests**: Test whether multiple functions work correctly together. 
- **Regression tests**: Ensure your code produces the same outputs as previous versions.



### Cool tip
You can include a cool badge to show visually whether your tests are passing or failing, like so

[![Tests](https://github.com/vboussange/rere/actions/workflows/runtest.yml/badge.svg)](https://github.com/vboussange/rere/actions/workflows/runtest.yml)

Cool right?


# Conclusion

Reproducibility of computational work is increasingly valued. 
Julia (and GitHub) offer several tools (listed in this section) to facilitate implementing good reproduciblity, so take advantage of them. It might require some ajustments in your workflow, but they will definitely be worth it.


## Launching scripts from the command line
Specially if you are doing heavy work, that runs for longer times, using a shell script to run your Julia code might be better than using a notebook. Here is what this script could look like

```sh
#!/bin/bash
date
echo "lauching script"
julia --project=. --threads 1 my-analysis.jl &> "stdout/my-analysis.out"
wait
echo "computation over"
date
```

If this script is in a file called `run_my_analysis.sh`, you first need to make the file executable

```sh
chmod u+x run_my_analysis.sh
```

Once this is done, just run
```sh
./run_my_analysis.sh
```

And you'll get as output
```
Thu Mar 23 22:45:18 CET 2023
lauching script
computation over
Thu Mar 23 22:45:30 CET 2023
```


# Your turn!

- 💻[Exercise: Set up and play with your first Julia project!](exercise23.md) 

# More resources
- [A multi-language overview on how to test your research project code](https://vboussange.github.io/post/testing-your-research-code/)
- [Julia documentation on unit testing](https://docs.julialang.org/en/v1/stdlib/Test/)
- [Modern Julia Workflows](https://modernjuliaworkflows.org)
- [More on test-oriented development](https://julia.quantecon.org/software_engineering/testing.html)